In [1]:
import pandas as pd
from tkinter import filedialog
import tkinter as tk
from datetime import datetime
from tqdm import tqdm

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
token = "5BmOHyI_I2gLnzKgwlqh9TwP3OKW8zGqwrKc9cwvl1PWSLLWt2Sz93JT7aGytoQbRYjkQBg_HslAq8ZYLlMBew=="
org = "adata"
bucket = "imu"

client = InfluxDBClient(url="http://10.10.12.147:8086", token=token)

ModuleNotFoundError: No module named 'tqdm'

In [25]:
write_api = client.write_api(write_options=SYNCHRONOUS)

data = "gordon,host=host1,name=gordon used_percent=23.43234543"

write_api.write(bucket, org, data)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Platform-Error-Code': 'invalid', 'Date': 'Wed, 24 Feb 2021 06:54:04 GMT', 'Content-Length': '112'})
HTTP response body: {"code":"invalid","message":"unable to parse 'gordon,host=host1,name=gordon used_percent=nooo': invalid number"}


In [8]:
file1 = filedialog.askopenfilename()
rawdata = pd.read_csv(file1, sep=",",  skip_blank_lines=True)#inputfile

In [10]:
rawdata = rawdata.drop("Unnamed: 0", axis=1)

In [11]:
rawdata

,Acceleration_in_X,Acceleration_in_Y,Acceleration_in_Z,Roll,Pitch,Yaw,Lane_change,turning,braking,acceleration
0,-0.005,0.008,0.018,-1.523,0.015,0.012,0.0,0.0,0.0,0.0
1,0.016,-0.002,0.018,-1.522,0.012,0.012,0.0,0.0,0.0,0.0
2,0.037,-0.005,0.018,-1.520,0.014,0.011,0.0,0.0,0.0,0.0
3,0.038,-0.009,0.024,-1.523,0.014,0.011,0.0,0.0,0.0,0.0
4,0.012,-0.016,0.032,-1.525,0.012,0.011,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
301526,0.012,-0.008,0.046,-1.486,-0.027,1.616,0.0,0.0,1.0,0.0
301527,0.018,-0.007,0.038,-1.488,-0.028,1.613,0.0,0.0,1.0,0.0
301528,0.010,-0.001,0.029,-1.490,-0.031,1.610,0.0,0.0,1.0,0.0
301529,-0.007,0.005,0.025,-1.491,-0.033,1.608,0.0,0.0,1.0,0.0


In [34]:
x = 0
y = 0 
z = 0
Roll = 0
Pitch = 0
Yaw = 0
behavior = ""
for i in tqdm(range(len(rawdata))):
    x = rawdata["Acceleration_in_X"][i]
    y = rawdata["Acceleration_in_Y"][i]
    z = rawdata["Acceleration_in_Z"][i]
    Roll = rawdata["Roll"][i]
    Pitch = rawdata["Pitch"][i]
    Yaw = rawdata["Yaw"][i]
    if rawdata["Lane_change"][i] != 0:
        behavior = "lanchange"
    elif rawdata["braking"][i] != 0:
        behavior = "breaking"
    else:
        behavior = "normal"
    data = "vcu,x="+str(x)+",y="+str(y)+",z="+str(z)+",Roll="+str(Roll)+",pitch="+str(Pitch)+",Yaw="+str(Yaw)+",behavior="+str(behavior)+" datetime="+str(datetime.now().strftime("%f"))
    write_api.write(bucket, org, data)
        

100%|██████████████████████████████████████████████████████████████████████████| 301531/301531 [51:33<00:00, 97.47it/s]


In [30]:
data = str(48541154)+",x="+str(x)+",y="+str(y)+",z="+str(z)+",Roll="+str(Roll)+",pitch="+str(Pitch)+",Yaw="+str(Yaw)+",behavior="+str(behavior)+" datetime="+str(datetime.now().strftime("%f"))
data

'48541154,x=-0.006,y=0.015,z=0.019,Roll=-1.494,pitch=-0.034,Yaw=1.605,behavior=breaking datetime=509444'

In [31]:
write_api.write(bucket, org, data)